In [1]:
## Reading the training file
train_data = open('company_data_preprocessed.csv', 'r')
lines = train_data.readlines()
train_data.close()

In [2]:
## Let's check how the data looks like

lines[1]

'" ""Hoist Kredit AB is a Swedish financial services company. Hoist is active in nine European countries, and specializes in the acquisition and management of non-performing consumer loans. Hoist manages approximately 4 million receivables with a total aggregate face value of approximately EUR 9.5 billion and gross collections amounted to EUR 730 million in 2014. Hoist was previously listed on the Stockholm Stock Exchange but was de-listed in 2004. In 2003 the company divested its operations in non-performing consumer loans in Sweden but has since 2009 offered deposit accounts in Sweden, and now has a total deposit base of SEK 8 billion. In 2012 Hoist was one of the largest acquirers of non-performing consumer loans in Europe with an acquisition volume of approximately SEK 2 billion ,"" one significant acquisition being the acquisition of the UK debt-collection agency Robinson Way Ltd. On October 2013 Hoist listed a 10-year bond for SEK 350 million on Nasdaq OMX Stockholm."""""",,,,,,,

In [3]:
lines = [word.lower().strip('\s+') for word in lines]
lines[1]

'" ""hoist kredit ab is a swedish financial services company. hoist is active in nine european countries, and specializes in the acquisition and management of non-performing consumer loans. hoist manages approximately 4 million receivables with a total aggregate face value of approximately eur 9.5 billion and gross collections amounted to eur 730 million in 2014. hoist was previously listed on the stockholm stock exchange but was de-listed in 2004. in 2003 the company divested its operations in non-performing consumer loans in sweden but has since 2009 offered deposit accounts in sweden, and now has a total deposit base of sek 8 billion. in 2012 hoist was one of the largest acquirers of non-performing consumer loans in europe with an acquisition volume of approximately sek 2 billion ,"" one significant acquisition being the acquisition of the uk debt-collection agency robinson way ltd. on october 2013 hoist listed a 10-year bond for sek 350 million on nasdaq omx stockholm."""""",,,,,,,

In [4]:
# removing punctuations
import string
def remove_punctuation(text):  
    punct_remove = [punct for punct in str(text) if punct not in string.punctuation]
    punct_remove = ''.join(punct_remove)
    return punct_remove


In [46]:
text_list = []
for i in range(0,len(lines)):
    text_list.append(remove_punctuation(lines[i]))

with open('company_file_preprocessed_small.txt', 'w') as f:
    for item in text_list:
        f.write("%s" % item)

print(text_list[1])

 hoist kredit ab is a swedish financial services company hoist is active in nine european countries and specializes in the acquisition and management of nonperforming consumer loans hoist manages approximately 4 million receivables with a total aggregate face value of approximately eur 95 billion and gross collections amounted to eur 730 million in 2014 hoist was previously listed on the stockholm stock exchange but was delisted in 2004 in 2003 the company divested its operations in nonperforming consumer loans in sweden but has since 2009 offered deposit accounts in sweden and now has a total deposit base of sek 8 billion in 2012 hoist was one of the largest acquirers of nonperforming consumer loans in europe with an acquisition volume of approximately sek 2 billion  one significant acquisition being the acquisition of the uk debtcollection agency robinson way ltd on october 2013 hoist listed a 10year bond for sek 350 million on nasdaq omx stockholm



In [74]:
text=(open("company_file_preprocessed_small.txt").read())

In [75]:
chars = sorted(list(set(text)))
print('total chars: ', len(chars))
print(chars[0])

total chars:  38




In [76]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
n_chars = len(text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  6721204
Total Vocab:  38


In [77]:
import numpy as np
import random

maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [78]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [79]:
from keras.callbacks import LambdaCallback
import sys
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(100):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [80]:
from keras.callbacks import ModelCheckpoint

filepath = "weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss',
                             verbose=1, save_best_only=True,
                             mode='min')

In [ ]:
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=1, min_lr=0.001)
callbacks = [print_callback, checkpoint, reduce_lr]

In [ ]:
model.fit(x, y, batch_size=128, epochs=5, callbacks=callbacks)

Epoch 1/5
2240388/2240388 [==============================] - 2568s 1ms/step - loss: 2.7596

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "to aol video search
 beijing capital int"
to aol video search
 beijing capital intern the a comio a com

/Users/shalu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  


ions and an a buai of south commerse the compaind compan was a coal an a oolio 
----- diversity: 0.5
----- Generating with seed: "to aol video search
 beijing capital int"
to aol video search
 beijing capital intern the a comio a sero coi the to a soai  of the lideratio company are a set a rong compai a fo the 
----- diversity: 1.0
----- Generating with seed: "to aol video search
 beijing capital int"
to aol video search
 beijing capital intire botoi sbh i ioccgiemo and on ookh a industish for thou ioning a ndbagoperlis a a iowlio caindia 
----- diversity: 1.2
----- Generating with seed: "to aol video search
 beijing capital int"
to aol video search
 beijing capital intern ruso and o io 4 to r a roo wea avea enih progiation ond ode tuksh ly whio baiisticeu oo heldfoom

Epoch 00001: loss improved from inf to 2.75965, saving model to weights.hdf5
Epoch 2/5
2240388/2240388 [==============================] - 2483s 1ms/step - loss: 4.6001

----- Generating text after Epoch: 1
----- diversit

In [ ]:
def generate_text(length, diversity):
    # Get random starting text
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    for i in range(length):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char
    return generated

In [ ]:
print(generate_text(500, 0.2)